![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### ODPi Egeria Hands-On Lab
# Welcome to the Building a Data Catalog Lab

## Introduction

ODPi Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogues and platforms together so they can share information about data and technology (called metadata).

In this hands-on lab you will get a chance to work with three ODPi Egeria metadata servers to build a distributed catalog of data assets and then experiment with attaching feedback (comments) to the catalog entries from different servers.

## The Scenario

The ODPi Egeria team use the personas from the fictitious company called Coco Pharmaceuticals.  (See https://opengovernance.odpi.org/coco-pharmaceuticals/ for more information).

The two main character engaged in this scenario are Peter Profile and Erin Overview.

![Peter and Erin](../images/peter-and-erin.png)

In [44]:
petersUserId = "peterprofile"
erinsUserId  = "erinoverview"

Peter and Erin are cataloguing new data sets that have been received from a hospital.  These data sets are part of a clinical trial that the hospital is participating in.

## Setting up

Coco Pharmaceuticals make widespread use of ODPi Egeria for tracking and managing their data and related assets.
Figure 1 below shows their metadata servers and the platforms that are hosting them.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

In [45]:
import os

corePlatformURL     = os.environ.get('corePlatformURL','http://localhost:8080') 
dataLakePlatformURL = os.environ.get('dataLakePlatformURL','http://localhost:8081') 
devPlatformURL      = os.environ.get('devPlatformURL','http://localhost:8082')

Peter is using the data lake operations metadata server called `cocoMDS1`. This server is hosted on the Data Lake OMAG Server Platform.

In [46]:
server1            = "cocoMDS1"
server1PlatformURL = dataLakePlatformURL

The following request checks that this server is running.

In [51]:
import requests
import pprint
import json

adminUserId = "garygeeke"

isServer1ActiveURL = server1PlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + server1 + "/status"

print (" ")
print ("GET " + isServer1ActiveURL)
print (" ")

response = requests.get(isServer1ActiveURL)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

serverStatus = response.json().get('active')
if serverStatus == True:
    print("Server " + server1 + " is active - ready to begin")
else:
    print("Server " + server1 + " is down - start it before proceeding")


 
GET http://localhost:8081/open-metadata/platform-services/users/garygeeke/server-platform/servers/cocoMDS1/status
 
Returns:
{
    "relatedHTTPCode": 200,
    "serverName": "cocoMDS1",
    "serverStartTime": "2019-09-17T16:01:00.105+0000",
    "active": true
}
 
Server cocoMDS1 is active - ready to begin


----
If you see `Server cocoMDS1 is active - ready to begin` then the server is running.  If the server is down, follow the instructions in the **Managing Servers** notebook to start the server.

----
## Exercise 1

### Adding assets to the catalog

In the first exercise, Peter Profile is adding some new data sets (assets) to the catalog. 

Peter uses the **Asset Owner** Open Metadata Access Service (OMAS) API to manage assets in the catalog.  All of the request for the Asset Owner OMAS begin with the following URL root.

In [52]:
server1AssetOwnerURL = server1PlatformURL + '/servers/' + server1 + '/open-metadata/access-services/asset-owner/users/' + petersUserId 

First Peter will query the current list of Clinical Trial Assets from cocoMDS1.

In [56]:

server1GetAssetsURL = server1AssetOwnerURL + '/assets/by-name?startFrom=0&pageSize=50'
searchString="Drop Foot"

print (" ")
print ("GET " + server1GetAssetsURL)
print ("{ " + searchString + " }")
print (" ")

response=requests.post(server1GetAssetsURL, data=searchString)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

if response.json().get('assets'):
    if len(response.json().get('assets')) == 1:
        print ("1 asset found")
    else:
        print (str(len(response.json().get('assets'))) + " assets found")
else:
    print ("No assets found")


 
GET http://localhost:8081/servers/cocoMDS1/open-metadata/access-services/asset-owner/users/peterprofile/assets/by-name?startFrom=0&pageSize=50
{ Drop Foot }
 
Returns:
{
    "class": "AssetsResponse",
    "relatedHTTPCode": 200,
    "startingFromElement": 0,
    "assets": [
        {
            "class": "Asset",
            "type": {
                "class": "ElementType",
                "elementTypeId": "2ccb2117-9cee-47ca-8150-9b3a543adcec",
                "elementTypeName": "CSVFile",
                "elementSuperTypeNames": [
                    "DataFile",
                    "DataStore",
                    "Asset",
                    "Referenceable"
                ],
                "elementTypeVersion": 1,
                "elementTypeDescription": "A description of a comma separated value (CSV) file",
                "elementSourceServer": "cocoMDS1",
                "elementOrigin": "LOCAL_COHORT",
                "elementHomeMetadataCollectionId": "4e5355ba-564a-4802-a

----
We can see here that no assets are returned as the repository is empty.

#### Adding weekly clinical trial assets


Peter is now going to create three weeks of clinical asset data. These are 3 data sets. We'll start with week 1

In [57]:
server1CreateAssetURL = server1AssetOwnerURL + '/assets/data-files/csv'

print (" ")
print ("POST: " + server1CreateAssetURL)

jsonHeader = {'content-type':'application/json'}
createAssetBody = {
	"class" : "NewCSVFileAssetRequestBody",
	"displayName" : "Week 1: Drop Foot Clinical Trial Measurements",
	"description" : "One week's data covering foot angle, hip displacement and mobility measurements.",
	"fullPath" : "file://secured/research/clinical-trials/drop-foot/DropFootMeasurementsWeek1.csv"
}

response=requests.post(server1CreateAssetURL, json=createAssetBody, headers=jsonHeader)

response.json()


 
POST: http://localhost:8081/servers/cocoMDS1/open-metadata/access-services/asset-owner/users/peterprofile/assets/data-files/csv


{'relatedHTTPCode': 400,
 'exceptionClassName': 'org.odpi.openmetadata.frameworks.connectors.ffdc.PropertyServerException',
 'exceptionErrorMessage': 'REPOSITORY-HANDLER-400-003 An unexpected error was returned by the property server during addCSVFileToCatalog request for open metadata access service Asset Owner OMAS on server cocoMDS1; message was REPOSITORY-HANDLER-404-009 Multiple FileFolder entities where found with a name of file:///secured: the identifiers of the returned entities are [aacc04e2-a2f1-4504-b49d-457efeee16de, 462058d8-7615-4540-8db4-9b831820ae30]; the calling method is addCSVFileToCatalog, the name parameter isd pathName and the server is cocoMDS1',
 'exceptionSystemAction': 'The system is unable to process the request.',
 'exceptionUserAction': 'Verify the access rights of the user.'}

----
Notice the response includes a property called “guids”.  This is the list of unique identifiers of the chain of assets for the folder structure and the file itself.  We need to save the file's unique identifier (the last one in the list) in a variable to use later.

In [34]:
asset1guids=response.json().get('guids')

for guid in asset1guids:
    asset1guid=guid

print (" ")
print ("The guid for asset 1 is: " + asset1guid)
print (" ")


Next GUID is ddce41af-5a1f-46b4-855b-66767cb7b521
Next GUID is f440c7e7-a0b0-4044-a390-408129f74302
Next GUID is f92f2704-15e0-4cca-9e1a-7d930d25644e
Next GUID is cb058d70-c88e-40cc-bc76-1a6c5a435704
Next GUID is 5cfb1b4f-d7ae-4bd0-8669-85f8787c6ee8
Next GUID is ddce41af-5a1f-46b4-855b-66767cb7b521
Next GUID is a6a3f1a2-4dbf-46b2-aeb6-ab10f06115fb
 
The guid for asset 1 is: a6a3f1a2-4dbf-46b2-aeb6-ab10f06115fb
 


----
Now let's take a look again at what assets are in the repository using the same get request we used earlier.


In [36]:

print (" ")
print ("GET " + server1GetAssetsURL)
print ("{ " + searchString + " }")
print (" ")

response=requests.post(server1GetAssetsURL, data=searchString)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

if response.json().get('assets'):
    if len(response.json().get('assets')) == 1:
        print ("1 asset found")
    else:
        print (str(len(response.json().get('assets'))) + " assets found")
else:
    print ("No assets found")


 
GET http://localhost:8081/servers/cocoMDS1/open-metadata/access-services/asset-owner/users/peterprofile/assets/by-name?startFrom=0&pageSize=50
{ file }
 
Returns:
{
    "class": "AssetsResponse",
    "relatedHTTPCode": 200,
    "startingFromElement": 0,
    "assets": [
        {
            "class": "Asset",
            "type": {
                "class": "ElementType",
                "elementTypeId": "229ed5cc-de31-45fc-beb4-9919fd247398",
                "elementTypeName": "FileFolder",
                "elementSuperTypeNames": [
                    "DataStore",
                    "Asset",
                    "Referenceable"
                ],
                "elementTypeVersion": 1,
                "elementTypeDescription": "A description of a folder (directory) in a file system.",
                "elementSourceServer": "cocoMDS1",
                "elementOrigin": "LOCAL_COHORT",
                "elementHomeMetadataCollectionId": "4e5355ba-564a-4802-ac4f-1b74eb993078"
            

----

Peter is now going to add the next two weeks of assets

In [43]:

csvbody2 = {
	"class" : "NewCSVFileAssetRequestBody",
	"displayName" : "Week 2: Drop Foot Clinical Trial Measurements",
	"description" : "One week's data covering foot angle, hip displacement and mobility measurements.",
	"fullPath" : "file://secured/research/clinical-trials/drop-foot/DropFootMeasurementsWeek2.csv"
}

response2=requests.post(server1CreateAssetURL, json=csvbody2, headers=jsonHeader)

print ("Second request responded with: " + str(response2.status_code))

asset2guids=response2.json().get('guids')

for guid in asset2guids:
    asset2guid=guid


csvbody3 = {
	"class" : "NewCSVFileAssetRequestBody",
	"displayName" : "Week 3: Drop Foot Clinical Trial Measurements",
	"description" : "One week's data covering foot angle, hip displacement and mobility measurements.",
	"fullPath" : "file://secured/research/clinical-trials/drop-foot/DropFootMeasurementsWeek3.csv"
}

response3=requests.post(server1CreateAssetURL, json=csvbody3, headers=jsonHeader)

print ("Third request responded with: "  + str(response3.status_code))

asset3guids=response3.json().get('guids')

for guid in asset3guids:
    asset3guid=guid

print (" ")
print ('Asset 1 guid is: ' + asset1guid)
print ('Asset 2 guid is: ' + asset2guid)
print ('Asset 3 guid is: ' + asset3guid)


Second request responded with: 200


ConnectionError: HTTPConnectionPool(host='localhost', port=8081): Max retries exceeded with url: /servers/cocoMDS1/open-metadata/access-services/asset-owner/users/peterprofile/assets/data-files/csv (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa9600290f0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [42]:
print (" ")
print ('Asset 1 guids is: ' + str(asset1guids))
print ('Asset 2 guids is: ' + str(asset2guids))
print ('Asset 3 guids is: ' + str(asset3guids))

 
Asset 1 guids is: ['ddce41af-5a1f-46b4-855b-66767cb7b521', 'f440c7e7-a0b0-4044-a390-408129f74302', 'f92f2704-15e0-4cca-9e1a-7d930d25644e', 'cb058d70-c88e-40cc-bc76-1a6c5a435704', '5cfb1b4f-d7ae-4bd0-8669-85f8787c6ee8', 'ddce41af-5a1f-46b4-855b-66767cb7b521', 'a6a3f1a2-4dbf-46b2-aeb6-ab10f06115fb']
Asset 2 guids is: ['d0442a52-98ab-40ee-ad27-290bc6bec544', '93e73568-14d2-429a-8f45-9adad3553fc7', '10cf04b7-48b8-4daf-90f8-7b38554abb52', '94e8dcdb-dc92-4bbd-937f-8034c2727784', 'a92be820-b2dc-4380-8006-345d975c4852', 'd0442a52-98ab-40ee-ad27-290bc6bec544', '97e11745-2217-4ed3-9162-2778a97e88fa']
Asset 3 guids is: ['346aba8f-4f2a-44f6-a4fb-e645693f9459', '0ea68beb-78c0-4036-baac-b790de9e0e23', '45c24948-d157-44d3-812a-ccb0c796d90a', '8cf520b2-d380-4d2c-95f1-c7ffdd3037d1', '3f255553-e519-4d5c-923e-72eac74af0bb', '346aba8f-4f2a-44f6-a4fb-e645693f9459', '5f1a63c8-16d9-47c9-a0e8-95c1420429e7']


----
Peter has successfully onboarded three file assets:

In [ ]:

print (" ")
print ("GET " + server1GetAssetsURL)
print ("{ " + searchString + " }")
print (" ")

response=requests.post(server1GetAssetsURL, data=searchString)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

if response.json().get('assets'):
    if len(response.json().get('assets')) == 1:
        print ("1 asset found")
    else:
        print (str(len(response.json().get('assets'))) + " assets found")
else:
    print ("No assets found")
    

----
## Exercise 2 - Sharing the catalog and adding feedback

In this next exercise Erin is going to work with the assets that Peter created.  Erin is part of the governance team.  She is accessing
metadata using the `cocoMDS2` server.  It sits on the core OMAG Server Platform.

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms (repeat)

In [ ]:
server2            = "cocoMDS2"
server2PlatformURL = corePlatformURL

This next code checks that cocoMDS2 is running ...

In [ ]:

isServer2ActiveURL = server2PlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + server2 + "/status"

print (" ")
print ("GET " + isServer2ActiveURL)
print (" ")

response = requests.get(isServer2ActiveURL)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

serverStatus = response.json().get('active')
if serverStatus == True:
    print("Server " + server2 + " is active - ready to begin")
else:
    print("Server " + server2 + " is down - start it before proceeding")


----
If you see Server cocoMDS2 is active - ready to begin then the server is running. If the server is down, follow the instructions in the **Managing Servers** notebook to start the server.

----
The metadata servers `cocoMDS1` and `cocoMDS2` are part of the same open metadata cohort called `cocoCohort`.  This means that they are actively sharing metadata.

![Figure 2](../images/coco-pharmaceuticals-systems-metadata-servers.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

----
Even though Erin is connected to a different server to Peter, she can see the same assets.

In [ ]:

server2AssetConsumerURL = server2PlatformURL + '/servers/' + server2 + '/open-metadata/access-services/asset-consumer/users/' + erinsUserId 
server2GetAssetsURL = server2AssetConsumerURL + '/assets/by-name?startFrom=0&pageSize=50'

print (" ")
print ("GET " + server2GetAssetsURL)
print ("{ " + searchString + " }")
print (" ")

response=requests.post(server2GetAssetsURL, data=searchString)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

if response.json().get('assets'):
    if len(response.json().get('assets')) == 1:
        print ("1 asset found")
    else:
        print (str(len(response.json().get('assets'))) + " assets found")
else:
    print ("No assets found")


----
Erin looks at the new assets that Peter has defined and has a question.  She adds a comment to the first asset.

In [ ]:

server2AddCommentURL = server2AssetConsumerURL + '/assets/' + asset1guid + '/comments'

print("")
print ("POST " + server2AddCommentURL)

commentBody={
	"class" : "CommentRequestBody",
	"commentType" : "QUESTION",
	"commentText" : "This file has much less data than normal.  Did the hospital provide any additional information about this batch to explain it?",
    "isPublic" : True
}
addCommentResponse = requests.post(server2AddCommentURL, json=commentBody, headers=jsonHeader)

addCommentResponse.json()

In [ ]:
commentGUID = addCommentResponse.json().get('guid')

print (" ")
print ('Erin\'s comment guid is: ' + commentGUID)

----
The comment is attached to the asset.  Peter can query an asset's comments as follows:

In [ ]:

server1ConnectedAssetURL = server1PlatformURL + '/servers/' + server1 + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + petersUserId 
server1CommentQuery = server1ConnectedAssetURL + '/assets/' + asset1guid + '/comments?elementStart=0&maxElements=50'

print (" ")
print ("GET " + server1CommentQuery)

getCommentsResponse = requests.get(server1CommentQuery)
getCommentsResponse.json()


----
He replies to Erin's question

In [ ]:

server1AssetConsumerURL = server1PlatformURL + '/servers/' + server1 + '/open-metadata/access-services/asset-consumer/users/' + petersUserId 
server1CommentReplyURL = server1AssetConsumerURL + '/assets/' + asset1guid + '/comments/' + commentGUID + '/replies'

print (" ")
print ("POST " + server1CommentReplyURL)

commentReplyBody={
	"class" : "CommentRequestBody",
	"commentType" : "ANSWER",
	"commentText" : "I checked back with Bobbie Records and they had an air conditioning failure that caused them to cancel patient appointments for 2 days - hence less data.  They are working to catch up on their waiting list so expect increased data for the next few weeks.",
    "isPublic" : True
}

addCommentReplyResponse = requests.post(server1CommentReplyURL, json=commentReplyBody, headers=jsonHeader)
addCommentReplyResponse.json()

----
Erin views the reply.

In [ ]:
server2ConnectedAssetURL = server2PlatformURL + '/servers/' + server2 + '/open-metadata/common-services/asset-consumer/connected-asset/users/' + erinsUserId 
server2CommentReplyQuery = server2ConnectedAssetURL + '/assets/' + asset1guid + '/comments/' + commentGUID + '/replies?elementStart=0&maxElements=50'

print (" ")
print ("GET " + server2CommentReplyQuery)

getCommentRepliesResponse = requests.get(server2CommentReplyQuery)

getCommentRepliesResponse.json()

----
This is the current information known about the first asset:

In [ ]:
server2GetAsset1 = server2ConnectedAssetURL + '/assets/' + asset1guid

print (" ")
print ("GET " + server2GetAsset1)

getAssetResponse = requests.get(server2GetAsset1)

getAssetResponse.json()

In [ ]:
server2GetRelatedAssets1 = server2ConnectedAssetURL + '/assets/' + asset1guid + '/related-assets?elementStart=0&maxElements=50'

print (" ")
print ("GET " + server2GetRelatedAssets1)

getAssetResponse = requests.get(server2GetRelatedAssets1)

getAssetResponse.json()

## Summary of Exercise 1 and 2

In the first two exercises of this hands-on lab you have shown that two servers with their own repositories can share and extend the metadata contributed by the other.  It began by Peter creating three assets in cocoMDS1.  Erin then connected to cocoMDS2 and she could also see these assets.  Then Erin was able to attach a comment to one of those assets through cocoMDS2 and Peter was then able to response through cocoMDS1.

Hence this is a truly distributed catalogue.


![Figure 3](../images/distributed-asset-with-comments.png)
> **Figure 3:** Asset and Comments distributed across 2 servers


----
## Exercise 3 - controlling access to assets

In the next exercise we will consider how organizations control the visability of assets.
Peter and Erin are joined by their colleague Callie Quartile, a data scientist working in the research team.

![Callie Quartile](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/callie-quartile.png)

Callie's userId is `calliequartile`.

In [ ]:
calliesUserId = 'calliequartile'

----
## Bonus material

This final section is an opportunity to dig a little deeper into the workings of Egeria.

The APIs used in the exercises above are from the access services - or Open Metadata Access Services (OMASs) to give them their formal name.  These APIs are domain specific - designed to use by tools, engines and platforms.

Underneath the access services are the repository services (Open Metadata Repository Services (OMRS)) and the platform services (Open Metadata and Governance (OMAG) Server Platform Services).

The repository services manage the exchange of metadata between servers.  The platform services provide a platform for running Egeria servers such as cocoMDS1 and cocoMDS2.


### Repository services

The repository services provide the ability for metadata to be accessed and exchanged from different servers.
Each server that has a repository (store) of metadata is assigned a **metadata collection id**.  This is a unique identifer that is associated with all metadata that originates from that repository.

The command below extracts the metadata collection id for cocoMDS1.

In [ ]:
server1RepositoryServicesURL = server1PlatformURL + '/servers/' + server1 + '/open-metadata/repository-services/users/' + adminUserId 
server1MetadataColectionIdQuery = server1RepositoryServicesURL + '/metadata-collection-id'

print (" ")
print ("GET " + server1MetadataColectionIdQuery)

response = requests.get(server1MetadataColectionIdQuery)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

serverStatus = response.json().get('relatedHTTPCode')
if serverStatus == 200:
    cocoMDS1MetadataCollectionId = response.json().get('metadataCollectionId')
    print("Metadata collection id for " + server1 + " is " + cocoMDS1MetadataCollectionId)
else:
    print("Server " + server1 + " is not able to supply a metadata collection id")

----
Now we extract the metadata collection id for cocoMDS2.

In [ ]:
server2RepositoryServicesURL = server2PlatformURL + '/servers/' + server2 + '/open-metadata/repository-services/users/' + adminUserId 
server2MetadataColectionIdQuery = server2RepositoryServicesURL + '/metadata-collection-id'

print (" ")
print ("GET " + server2MetadataColectionIdQuery)

response = requests.get(server2MetadataColectionIdQuery)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

serverStatus = response.json().get('relatedHTTPCode')
if serverStatus == 200:
    cocoMDS2MetadataCollectionId = response.json().get('metadataCollectionId')
    print("Metadata collection id for " + server2 + " is " + cocoMDS2MetadataCollectionId)
else:
    print("Server " + server2 + " is not able to supply a metadata collection id")

----

The metadata collection id is allocated when the server is first configured.  Once the server starts sharing metadata, the metadata collection id must never change as it is used in the metadata repository to identify where each piece of metadata came from.

The cocoMDS4 server does not have a repository and uses federated queries to retrieve metadata from other servers.

In [ ]:
server4            = "cocoMDS4"
server4PlatformURL = dataLakePlatformURL

server4RepositoryServicesURL = server4PlatformURL + '/servers/' + server4 + '/open-metadata/repository-services/users/' + adminUserId 
server4MetadataColectionIdQuery = server4RepositoryServicesURL + '/metadata-collection-id'

print (" ")
print ("GET " + server4MetadataColectionIdQuery)

response = requests.get(server4MetadataColectionIdQuery)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

serverStatus = response.json().get('relatedHTTPCode')
if serverStatus == 200:
    cocoMDS2MetadataCollectionId = response.json().get('metadataCollectionId')
    print("Metadata collection id for " + server4 + " is " + cocoMDS4MetadataCollectionId)
else:
    print("Server " + server4 + " is not able to supply a metadata collection id")

----
This result is also a demonstration of the error handling in Egeria. All errors consist of a message, system action and user response.

----
Metadata instances such as the Assets and Comments that you were working with in Exercises 1 and 2 are stored in the repository as entities.  These entities are linked together with relationships (it is a logical graph model).

The command below uses the respository services to retrieve one of the assets created in exercise 1

In [ ]:
server2AssetEntityQuery = server2RepositoryServicesURL + '/instances/entity/' + asset1guid

print (" ")
print ("GET " + server2AssetEntityQuery)

response = requests.get(server2AssetEntityQuery)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

The entity includes its type definition and the properties of the asset.  Also notice the metadata collection id for cocoMDS1 around the middle of the structure.

Contrast the asset entity with the comment that Erin created.  Notice the type information is different, and the metadata collection id for cocoMDS2.

In [ ]:
server2CommentEntityQuery = server2RepositoryServicesURL + '/instances/entity/' + commentGUID

print (" ")
print ("GET " + server2CommentEntityQuery)

response = requests.get(server2CommentEntityQuery)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

----
Finally, consider the relationship between the asset and the comment.  It includes summary information about the two entities (called an **entity proxy**).  This is how it is possible to transmit and even store relationships independently of the entities.

In [ ]:
server2AssetRelationshipQuery = server2RepositoryServicesURL + '/instances/entity/' + asset1guid + '/relationships'

print (" ")
print ("POST " + server2AssetRelationshipQuery)

relationshipRequestBody={
	"class" : "TypeLimitedFindRequest",
	"offset" : "0",
	"pageSize" : "100" 
}
response = requests.post(server2AssetRelationshipQuery, json=relationshipRequestBody, headers=jsonHeader)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")



Which server was the relationship created in?

----
#### Open Metadata Cohorts

The metadata exchange between the servers is a peer-to-peer protocol.  Each server registers with one or more open metadata cohorts.  

Figure 4 shows which metadata servers belong to each cohort.

![Figure 4](../images/coco-pharmaceuticals-systems-metadata-servers.png)
> **Figure 4:** Membership of Coco Pharmaceuticals' cohorts

----
The command below queries cocoMDS2's view of the cohorts

In [ ]:
server2cohortURLcore =  server2RepositoryServicesURL + '/metadata-highway'

import pprint
import json

print (" ")
print ("Querying cohorts for " + server2 + " ...")
url = server2cohortURLcore + '/cohort-descriptions'
print ("GET " + url)

response = requests.get(url)

print (" ")

serverStatus = response.json().get('relatedHTTPCode')
if serverStatus == 200:
    cohorts = response.json().get('cohorts')
    cohort1 = cohorts[0]
    cohort1Name = cohort1.get('cohortName')
    print("Cohort 1 for " + server2 + " is " + cohort1Name)
    cohort2 = cohorts[1]
    cohort2Name = cohort2.get('cohortName')
    print("Cohort 2 for " + server2 + " is " + cohort2Name)
    cohort3 = cohorts[2]
    cohort3Name = cohort3.get('cohortName')
    print("Cohort 3 for " + server2 + " is " + cohort3Name)
else:
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")

----
There are more examples and explanation about the way that the cohorts work in the **Understanding Cohorts** notebook.


----
### Metadata security

Security of metadata is extremely important.  Egeria has multiple levels of security so that access to individual metadata instances can be controlled.  The command below is a simple test when an unauthorized user tries to access one of Coco Pharmaceutical metadata servers.


In [ ]:
unauthorizedUserQuery = server2PlatformURL + '/servers/' + server2 + '/open-metadata/repository-services/users/evilEdna/metadata-collection-id'

print (" ")
print ("GET " + unauthorizedUserQuery)

response = requests.get(unauthorizedUserQuery)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

----
### Platform services

The platform services are for the infrastructure team running an Egeria service.  In the case of a cloud service, this may be a different organization to the metadata owners.  As a result, there is a separation of users able to work with the platform services verses the access and repository services.

This first command queries the servers running on a platform.

In [ ]:
corePlatformServices = corePlatformURL + '/open-metadata/platform-services/users/' + adminUserId + '/server-platform'
corePlatformServers  = corePlatformServices + '/servers'

print (" ")
print ("CorePlatform's Servers ")
print ("GET " + corePlatformServers)

response = requests.get(corePlatformServers)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

dataLakePlatformServices = dataLakePlatformURL + '/open-metadata/platform-services/users/' + adminUserId + '/server-platform'
dataLakePlatformServers  = dataLakePlatformServices + '/servers'

print (" ")
print ("DataLakePlatform's Servers ")
print ("GET " + dataLakePlatformServers)

response = requests.get(dataLakePlatformServers)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")

----
This last command queries the services active on server 1

In [ ]:
server1Services = dataLakePlatformServices + '/servers/' + server1 + '/services'

print (" ")
print (server1 + " services ")
print ("GET " + server1Services)

response = requests.get(server1Services)

print ("Returns:")
prettyResponse = json.dumps(response.json(), indent=4)
print (prettyResponse)
print (" ")